In [7]:
import time
import requests
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 2Captcha API Key
API_KEY = "48458d297de753aa014492988c0d4acc"

# Website URL
url = 'https://excise.gos.pk/vehicle/vehicle_search' # Replace with the target URL
site_key = "6LeAAg4UAAAAACaH6ib9TkLUHyXma2UUR5u5BVVX"  # Replace with the site's reCAPTCHA key (from page source)
# website_url = 
# sitekey="6LeAAg4UAAAAACaH6ib9TkLUHyXma2UUR5u5BVVX"
# captcha_response = captcha_solver(website_url, sitekey)
# key  = "48458d297de753aa014492988c0d4acc"
# Function to solve reCAPTCHA
def solve_recaptcha(api_key, site_key, url):
    # Step 1: Submit a reCAPTCHA solving request to 2Captcha
    captcha_url = "http://2captcha.com/in.php"
    payload = {
        'key': api_key,
        'method': 'userrecaptcha',
        'googlekey': site_key,
        'pageurl': url,
        'json': 1
    }
    response = requests.post(captcha_url, data=payload)
    request_result = response.json()

    if request_result["status"] != 1:
        raise Exception(f"Failed to submit CAPTCHA: {request_result['request']}")

    captcha_id = request_result["request"]

    # Step 2: Poll for CAPTCHA solution
    result_url = f"http://2captcha.com/res.php?key={api_key}&action=get&id={captcha_id}&json=1"
    for _ in range(30):  # Check every 5 seconds for up to 2 minutes
        result = requests.get(result_url).json()
        if result["status"] == 1:
            return result["request"]  # Return the CAPTCHA solution
        time.sleep(5)

    raise Exception("CAPTCHA solution timed out.")

# Use Selenium to interact with the webpage
def main():
    # Step 1: Open the website
    driver = webdriver.Chrome()  # Make sure chromedriver is in PATH
    driver.get(url)

    # Step 2: Solve the reCAPTCHA
    print("Solving CAPTCHA...")
    captcha_solution = solve_recaptcha(API_KEY, site_key, url)
    print("CAPTCHA Solved!")

    
    input_key = driver.find_element(By.ID, 'reg_no')
    input_key.send_keys("BWH135")
    
    # Step 3: Insert the CAPTCHA token
    driver.execute_script(f'document.getElementById("g-recaptcha-response").innerHTML="{captcha_solution}";')

    # Step 4: Submit the form (adjust the selector to match the site's form button)
    driver.find_element(By.ID, "search_veh").click()
    print("Form Submitted!")

    time.sleep(10)  # Wait to observe the result
    response  = driver.find_element(By.ID,"veh_res")
    time.sleep(2)
    print(response.text)
    replay =  response.text
    time.sleep(3)
    # Step 5: Close the browser
    driver.quit()
    return  replay

# if __name__ == "_main_":


In [8]:
res =  main()
if resp == 'Record Not found':
    reg_num , model , engin = reg_number ,None  , None
else:
    splited_val = resp.split("\n")[2].split(" ")[:-2]
    reg_num , model, engin =splited_val[0], " ".join(splited_val[1:-1]) , splited_val[-1]  

Solving CAPTCHA...
CAPTCHA Solved!
Form Submitted!
Online Vehicle Verification List
Registration No. Make Engine No. Action
BWH-135 HONDA CITY 2022 L12B51009916 View Details
